This notebook attempts to approach question 5.8 with a 3-layered HMM.

In [9]:
import numpy as np

In [302]:
alphabet = np.chararray(["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"])
N = np.size(alphabet)

TypeError: 'str' object cannot be interpreted as an integer

In [87]:
fnames = np.array(["david", "anton", "fred", "jim", "barry"])
snames = np.array(["barber", "ilsung", "fox", "chain", "fitzwilliam", "quinceadams", "grafvonunterhosen"])

In [285]:
states = np.array(["r1", "f1", "f2", "f3", "f4", "f5", "r2", "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8", "s9", "s10", "s11", "s12", "s13", "s14", "s15", "s16", "s17"])
K = np.size(states)

In [286]:
def probability_name_longer_than(names, n):
    """ Returns the probability of a name in the given names being longer
    than the given number of characters. """
    c = t = 0
    for name in names:
        if len(name) >= n:
            t += 1
            if len(name) > n:
                c += 1
    return c / t
        

ss = np.zeros((K, K))
for i, x in enumerate(states):
    for j, y in enumerate(states):
        if x == "r1":
            if y == "r1":
                ss[i][j] = 0.8
            elif y == "f1":
                ss[i][j] = 0.2
            else:
                ss[i][j] = 0.0
        elif x == "r2":
            if y == "r2":
                ss[i][j] = 0.8
            elif y == "s1":
                ss[i][j] = 0.2
            else:
                ss[i][j] = 0.0
        elif x.startswith("f"):
            if y.startswith("f") and int(x[1:]) + 1 == int(y[1:]):
                ss[i][j] = probability_name_longer_than(fnames, int(x[1:]))
            elif y == "r2":
                ss[i][j] = 1 - probability_name_longer_than(fnames, int(x[1:]))
            else:
                ss[i][j] = 0.0
        elif x.startswith("s"):
            if y.startswith("s") and int(x[1:]) + 1 == int(y[1:]):
                ss[i][j] = probability_name_longer_than(snames, int(x[1:]))
            elif y == "r1":
                ss[i][j] = 1 - probability_name_longer_than(snames, int(x[1:]))
            else:
                ss[i][j] = 0.0

In [287]:
vh = np.zeros((N, N))
for i, a in enumerate(alphabet):
    for j, b in enumerate(alphabet):
        vh[i][j] = (i == j) * 0.3 + (i != j) * 0.7 / 25

In [291]:
def count_names_with(names, letter, index):
    """ Returns the share of names having the given letter at the given index. """
    c = t = 0
    for name in names:
        if len(name) > index:
            t += 1
            c += name[index] == letter
    return c / t if c else c

def probability_pair(names, i, a, j, b):
    c = t = 0
    for name in names:
        if len(name) > j:
            if name[i] == a:
                t += 1
                c += (name[i] == a) and (name[j] == b)
    return c / t if c else c
    

hhs = np.zeros((N, N, K))
for i, a in enumerate(alphabet):
    for j, b in enumerate(alphabet):
        for k, state in enumerate(states):
            if state.startswith("r"):
                hhs[i][j][k] = 1 / 26
            elif state.startswith("f"):
                if state == "f1":
                    hhs[i][j][k] = count_names_with(fnames, b, 0)
                else:
                    x = int(state[1:]) - 2
                    y = x + 1
                    hhs[i][j][k] = probability_pair(fnames, x, a, y, b)
            elif state.startswith("s"):
                if state == "s1":
                    hhs[i][j][k] = count_names_with(snames, b, 0)
                else:
                    x = int(state[1:]) - 2
                    y = x + 1
                    hhs[i][j][k] = probability_pair(snames, x, a, y, b)                    

In [334]:
# def index_of(letter):
#     return (np.where(alphabet == letter))[0][0]

# noisy_sequence = np.array(tuple(open("sequence.txt", "r").read()))
# T = len(clean_sequence)
# clean_sequence = np.zeros(T, dtype="int")
# state_sequence = np.zeros(T, dtype="int")

# for k in range(1, T):
#     best_p, best_letter, best_state = 0, 0, 0
#     for i, s1 in enumerate(states):
#         for j, letter in enumerate(alphabet):
#             h0 = clean_sequence[k-1]
#             v1 = index_of(noisy_sequence[k])
#             s0 = int(state_sequence[k-1])
#             a = ss[s0][i]
#             b = vh[v1][j]
#             c = hhs[h0][j][i]
#             p = a * b * c
#             #print("State: {}, Character: {}, P: {}".format(s1, letter, p))
#             if p > best_p:
#                 best_p = p
#                 best_letter = j
#                 best_state = i
#     state_sequence[k] = best_state
#     clean_sequence[k] = best_letter
#     print(alphabet[best_letter], end="")